In [1]:
# imports
import yfinance as yf
from dateutil.relativedelta import relativedelta
import math
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import basinhopping
from scipy.stats import norm

In [2]:
# Initialise variables
stock_ticker = "AAPL"
interest_rate = 0.055 # Give as decimal, US Treasury interest rate
capital = 100000 # Initial capital in USD
no_data_available_log = []

end_date = dt.date.today()
start_date = end_date - relativedelta(months=1)

ticker = yf.Ticker(stock_ticker)
try:
    stock_data = ticker.history(start=start_date, end=end_date, auto_adjust=False, actions=False)
    stock_data = stock_data.sort_index(ascending=False)
except Exception as e:
    print(f"Error fetching stock data: {e}")
    stock_data = pd.DataFrame()

# Round to 4 decimal places as that is a great enough resolution for this project
# Also simplifies the capital calculations later on
stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close']] = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close']].round(3)

# Use latest adjusted close price for stock price
stock_price = stock_data['Adj Close'].iloc[0]

# Calculate nearest 100 multiple of assets that can be invested
assets_investable = (capital // stock_price) // 100 * 100

In [3]:
# Fetch options data
options_data = pd.DataFrame()

try:
    expiries = ticker.options
    for exps in expiries:
        
        opt = ticker.option_chain(exps)

        if opt.calls.empty and opt.puts.empty:  # Check if there's no data
            no_data_available_log.append(exps)  # Log the date
            continue  # Skip to the next date

        opt = pd.concat([opt.calls, opt.puts])
        opt['expirationDate'] = exps
        options_data = pd.concat([options_data, opt], ignore_index=True)

        options_data['expirationDate'] = pd.to_datetime(options_data['expirationDate'])
        # Calculate DTE as a decimal for increase accuracy for Greeks calculations
        options_data['DTE'] = (options_data['expirationDate'] - dt.datetime.today()).dt.days / 365

        # Boolean column if option is a call
        options_data['Call'] = options_data['contractSymbol'].str[4:].apply(
            lambda x: "C" in x
        )

        options_data[['bid', 'ask', 'strike']] = options_data[['bid', 'ask', 'strike']].apply(pd.to_numeric)
        options_data['mark'] = (options_data['bid'] + options_data['ask']) / 2 # Calc mid-point of bid-ask spread

        # Drop unnecessary columnds
        options_data = options_data.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])
        # Remaining: contractSymbol, strike, bid, ask, volume, openInterest, impliedVolatility, expirationDate, DTE, Call, mark

        # Remove options with an IV of less than 1% and filter to +-10% of current stock price
        options_data = options_data[(options_data['impliedVolatility'] > 0.01)
                                    & (options_data['strike'] > stock_price * 0.9)
                                    & (options_data['strike'] < stock_price * 1.1)]

except Exception as e:
    print(f"Error fetching options data: {e}")


# Sort the DataFrame by 'volume' in descending order
options_data = options_data.sort_values(by='volume', ascending=False)
# Calculate the number of rows that correspond to the top 10%
top_10_percent_count = int(len(options_data) * 0.1)

# Filter to top 10% by liquidity
options_data = options_data.head(top_10_percent_count)

In [4]:
def calculateGreeks(stock_price, strike_price, interest_rate, time_to_expiration, impliedVolatility, option_type='call'):

    # Calculate d1
    d1 = (math.log(stock_price / strike_price) + (interest_rate + 0.5 * impliedVolatility ** 2) * time_to_expiration) / (impliedVolatility * math.sqrt(time_to_expiration))

    # Calculate d2
    d2 = d1 - impliedVolatility * np.sqrt(time_to_expiration)

    if option_type.lower() == 'call':
        # Calculate delta for a call option
        delta = norm.cdf(d1)
    else:
        # Calculate delta for a put option
        delta = -norm.cdf(-d1)

    # Calculate gamma
    gamma = norm.pdf(d1) / (stock_price * impliedVolatility * math.sqrt(time_to_expiration))

    # Calculate vega, multiplied by 0.01 to convert to %
    vega = stock_price * norm.pdf(d1) * math.sqrt(time_to_expiration) * 0.01

    # Calculate theta
    if option_type.lower() == 'call':
        theta = (- (stock_price * norm.pdf(d1) * impliedVolatility) / (2 * math.sqrt(time_to_expiration)) - interest_rate * strike_price * math.exp(-interest_rate * time_to_expiration) * norm.cdf(d2)) / 365
    else:
        theta = (- (stock_price * norm.pdf(d1) * impliedVolatility) / (2 * math.sqrt(time_to_expiration)) + interest_rate * strike_price * math.exp(-interest_rate * time_to_expiration) * norm.cdf(-d2)) / 365

    # Calculate rho
    if option_type.lower() == 'call':
        rho = strike_price * time_to_expiration * math.exp(-interest_rate * time_to_expiration) * norm.cdf(d2) * 0.01
    else:
        rho = -strike_price * time_to_expiration * math.exp(-interest_rate * time_to_expiration) * norm.cdf(-d2) * 0.01
    
    return delta, gamma, vega, theta, rho

In [5]:
def getOptionGreeks(options_data, stock_price, interest_rate):
    """
    Calculates the Greeks for each option in the DataFrame and updates the DataFrame with these values.
    """

    # Iterate over the DataFrame rows
    for index, row in options_data.iterrows():
        # Extract necessary parameters for each option
        strike_price = row['strike']
        time_to_expiration = row['DTE'] # Leave as a fraction of a year
        impliedVolatility = row['impliedVolatility']
        option_type = 'call' if row['Call'] else 'put'

        # Calculate Greeks
        delta, gamma, vega, theta, rho = calculateGreeks(stock_price, strike_price, interest_rate, time_to_expiration, impliedVolatility, option_type)

        # Store the Greeks in the DataFrame
        options_data.at[index, 'delta'] = delta
        options_data.at[index, 'gamma'] = gamma
        options_data.at[index, 'vega'] = vega
        options_data.at[index, 'theta'] = theta
        options_data.at[index, 'rho'] = rho

    return options_data.reset_index(drop=True)

In [6]:
options_data = getOptionGreeks(options_data, stock_price, interest_rate)

call_options = options_data[options_data['Call']]
atm_option_index = np.abs(call_options['strike'] - stock_price).idxmin()
atm_option = call_options.loc[atm_option_index]
lots = assets_investable / 100

atm_delta = -atm_option['delta'] * lots * 100 # Negative as short
atm_gamma = -atm_option['gamma'] * lots * 100

In [7]:
def adjustTotalDelta(total_delta):

    # Reduce delta with asset positions
    if total_delta < -0.5 or total_delta > 0.5:
        # Find the closest edge of the range (-0.5 or 0.5)
        closest_edge = -0.5 if total_delta < 0 else 0.5
        # Calculate the difference needed to reach the closest edge
        difference = total_delta - closest_edge
        # The number of integers to add or subtract
        # Use ceil for positive difference and floor for negative difference
        num_assets = -math.ceil(difference) if difference > 0 else -math.floor(difference)
        
        total_delta += num_assets # Alter total_delta after entering position

        return total_delta, num_assets
    else:
        # total_delta is within the range, no adjustment needed
        return total_delta, 0


In [8]:
def enterPosition(num_assets, stock_price, capital, best_options_data, atm_option):
    capital_invested = ((num_assets * stock_price) + best_options_data['total_lot_bid_price'].sum() - atm_option['bid'] * 100).round(4)
    current_capital = capital - capital_invested.round(4)
    return capital_invested, current_capital

In [10]:
deltas = options_data['delta'].values * 100 # Each option represents 100 shares
gammas = options_data['gamma'].values * 100
bid_prices = options_data['bid'].values * 100

# Store delta and gamma values for plotting
delta_path = []
gamma_path = []

# Objective function: Minimise the absolute combined delta and gamma
def objective(multiples):
    int_multiples = np.round(multiples) # Ensuring multiples are integers
    total_delta = np.sum(deltas * int_multiples) + atm_delta
    total_gamma = np.sum(gammas * int_multiples) + atm_gamma
    return abs(total_delta) + abs(total_gamma) * 100 # Arbitrary coefficient to get better results

# Parameters
x0 = np.zeros(len(options_data)) # Initial guess
bounds = [(0, 10) for _ in range(len(options_data))] # Bounds to test
minimizer_kwargs = {"method": "SLSQP", "bounds": bounds} # Local minimisation function
N = 10 # Number of times to run the optimisation

logged_results = []

# Perform optimisation using Basin-Hopping
for _ in range(N):
    result = basinhopping(objective, x0, minimizer_kwargs=minimizer_kwargs, niter=1000, stepsize=0.5, T=0.4, niter_success=300)
    if result.success:
        optimal_multiples = np.round(result.x) # Optimal multiples as integers
        total_delta = np.sum(deltas * optimal_multiples) + atm_delta
        total_gamma = np.sum(gammas * optimal_multiples) + atm_gamma
        logged_results.append((optimal_multiples, total_delta, total_gamma))
    else:
        raise ValueError("No solution found")

# Choose the result with gamma closest to zero
best_result = min(logged_results, key=lambda x: abs(x[2]))  # Selecting the set with gamma closest to zero
optimal_multiples, best_delta, best_gamma = best_result

# Boolean mask and apply to dataframe
mask = optimal_multiples != 0
best_options_data = options_data[mask].copy()
best_options_data['multiple'] = optimal_multiples[mask]
best_options_data['total_lot_bid_price'] = best_options_data['bid'] * best_options_data['multiple'] * 100

total_delta, num_assets = adjustTotalDelta(best_delta)

capital_invested, capital_remaining = enterPosition(num_assets, stock_price, capital, best_options_data, atm_option)

# Print the best result
print("Options Delta:", best_delta)
print("Assets position:", num_assets)
print("Delta after asset position:", total_delta)
print("Options Gamma:", best_gamma)
print("Capital invested:", capital_invested)
print("Capital remaining:", capital_remaining)
print(best_options_data[['contractSymbol', 'multiple', 'total_lot_bid_price']])


Options Delta: 6.955134048522609
Assets position: -7
Delta after asset position: -0.04486595147739081
Options Gamma: 0.0016452805856204122
Capital invested: 3426.59
Capital remaining: 96573.41
         contractSymbol  multiple  total_lot_bid_price
0   AAPL240301C00185000       1.0                 33.0
1   AAPL240301C00182500       1.0                 92.0
2   AAPL240301P00180000       1.0                 86.0
4   AAPL240308C00185000       1.0                108.0
7   AAPL240308C00182500       1.0                202.0
10  AAPL240315C00185000       1.0                172.0
12  AAPL240621P00175000       1.0                500.0
14  AAPL240621C00180000       1.0               1080.0
15  AAPL240301P00195000       1.0               1370.0
16  AAPL240308C00180000       1.0                320.0
17  AAPL240301P00175000       1.0                 10.0
18  AAPL240308C00187500       1.0                 57.0
19  AAPL240308C00190000       1.0                 30.0
20  AAPL240308P00180000       1.0    